In [30]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [31]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [32]:

def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [33]:

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [34]:

log_interval = 10;
batch_size = 64;
test_batch_size = 64;

use_cuda = False; #not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(1); #args.seed
device = torch.device("cuda" if use_cuda else "cpu");
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {};

lr = 0.01;
momentum = 0.5;


In [35]:

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs
)


In [36]:

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs
)


In [37]:

model = Net().to(device);
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum);


In [39]:

epochs = 3
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.213460
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.170403
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.076579
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.867874
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.413479
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.000340
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.776303
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.459810
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.487046
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.438144
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.408856
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.461841
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.428373
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.399202
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.384034
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.298051
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.501322
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.524580
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.337594
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.039069
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.145114
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.102241
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.133701
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.136900
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.053786
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.055050
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.028133
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.071126
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.099568
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.030294
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.038315
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.025524
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.040020
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.189820
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.062186
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.042761
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.020408
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.066949
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.084051


In [56]:

from torch.autograd import Variable


single_loaded_img = test_loader.dataset.data[0]
#predict_x = predict_x.to(device)
#predict_x = Variable(predict_x)
#print(predict_x)

def predict_image(image):
    image_tensor = image.unsqueeze_(0)
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
    input = input.to(device)
    output = model(input)
    #return output
    #index = output.data.cpu().numpy().argmax()
    #return index

#out_predict = model(single_loaded_img)

out_predict = predict_image(single_loaded_img)
